<a href="https://colab.research.google.com/github/bartlomiejsiusta/mgu/blob/main/MGU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
!pip install foolbox==3.3.1
import foolbox
import eagerpy as ep


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train.astype(np.float32)
x_test.astype(np.float32)
x_train = x_train/255.0
x_test = x_test/255.0


# dzielenie danych treningowych na treningowe i walidacyjne
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]


# one hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
#plt.imshow(x_train[2])


print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_valid shape:", x_valid.shape, "y_valid shape:", y_valid.shape)
print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)


x_train shape: (45000, 32, 32, 3) y_train shape: (45000, 10)
x_valid shape: (5000, 32, 32, 3) y_valid shape: (5000, 10)
x_test shape: (10000, 32, 32, 3) y_test shape: (10000, 1)


In [3]:
from keras.callbacks import ModelCheckpoint


model = tf.keras.applications.ResNet50(weights=None, classes=10, input_shape=(32, 32, 3))
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
# model.summary()

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train, y_train, batch_size=256, epochs=2, verbose=1, validation_data=(x_valid, y_valid), callbacks=[checkpointer])



Epoch 1/2
176/176 [==============================] - ETA: 0s - loss: 1.9963 - accuracy: 0.3455
Epoch 00001: val_loss improved from inf to 3.26254, saving model to model.weights.best.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


176/176 [==============================] - 63s 252ms/step - loss: 1.9963 - accuracy: 0.3455 - val_loss: 3.2625 - val_accuracy: 0.1212
Epoch 2/2
176/176 [==============================] - ETA: 0s - loss: 1.6541 - accuracy: 0.4182
Epoch 00002: val_loss improved from 3.26254 to 2.93055, saving model to model.weights.best.hdf5
176/176 [==============================] - 40s 226ms/step - loss: 1.6541 - accuracy: 0.4182 - val_loss: 2.9306 - val_accuracy: 0.2394


In [4]:
# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])


#model.load_weights('model.weights.best.hdf5')
pre_trained_model = tf.keras.models.load_model('model.weights.best.hdf5')

foolbox_model = foolbox.models.TensorFlowModel(model = pre_trained_model,bounds = (0.0, 1.0))
# FGSM
attack_fgsm = foolbox.attacks.FGSM()
epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
]

images, labels = ep.astensors(*foolbox.samples(foolbox_model, dataset="cifar10", batchsize=16)) # przyklad z githuba

x_test = tf.convert_to_tensor(x_test, dtype=np.float32)
y_test = tf.convert_to_tensor(y_test, dtype=np.int64)
new_x_test, new_y_test = ep.astensors(x_test, y_test)

clean_acc = foolbox.accuracy(foolbox_model, new_x_test, new_y_test)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

raw_advs, clipped_advs, success = attack_fgsm(foolbox_model, images, labels, epsilons=epsilons)

print(success)
# for i in range(1,3):
#     image = new_x_test[i]
#     _, advs, success = attack_fgsm(foolbox_model, image, new_y_test[i], epsilons=epsilons)
#     plt.imshow(np.squeeze(x_test[i]))
#     plt.show()
#     plt.imshow(np.squeeze(adversarial))



Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
clean accuracy:  10.0 %
TensorFlowTensor(<tf.Tensor: shape=(13, 16), dtype=bool, numpy=
                 array([[ True, False,  True,  True,  True,  True,  True,  True,  True,
                          True,  True,  True,  True,  True,  True,  True],
                        [ True, False,  True,  True,  True,  True,  True,  True,  True,
                          True,  True,  True,  True,  True,  True,  True],
                        [ True, False,  True,  True,  True,  True,  True,  True,  True,
                          True,  True,  True,  True,  True,  True,  True],
                        [ True, False,  True,  True,  True,  True,  True,  True,  True,
                          True,  True,  True,  True,  True,  True,  True],
                        [ True, False,  True,  True,  True,  True,  True,  True,  True,
                          True,  True,  True,  True,  True,  True,  True],
                

(16,)
